In [0]:
%sh curl --remote-name-all 'https://files.training.databricks.com/assessments/cse-take-home/{covertype,kafka,treecover,u.data,u.item}.csv'


[1/5]: https://files.training.databricks.com/assessments/cse-take-home/covertype.csv --> covertype.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   133  100   133    0     0   1528      0 --:--:-- --:--:-- --:--:--  1546

[2/5]: https://files.training.databricks.com/assessments/cse-take-home/kafka.csv --> kafka.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  135k  100  135k    0     0  4110k      0 --:--:-- --:--:-- --:--:-- 4110k

[3/5]: https://files.training.databricks.com/assessments/cse-take-home/treecover.csv --> treecover.csv
  % Total    % Received % Xferd  Average Speed   Time    Time 

In [0]:
dbutils.fs.cp("file:/databricks/driver/covertype.csv", "dbfs:/FileStore/tmp/covertype.csv")
dbutils.fs.cp("file:/databricks/driver/kafka.csv", "dbfs:/FileStore/tmp/kafka.csv")
dbutils.fs.cp("file:/databricks/driver/treecover.csv", "dbfs:/FileStore/tmp/treecover.csv")
dbutils.fs.cp("file:/databricks/driver/u.data.csv", "dbfs:/FileStore/tmp/u.data.csv")
dbutils.fs.cp("file:/databricks/driver/u.item.csv", "dbfs:/FileStore/tmp/u.item.csv")

Out[14]: True

In [0]:
treeCoverDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/tmp/treecover.csv")


In [0]:
treeCoverDF.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Cover_Type: integer (nullable = true)
 |-- Soil_Type: integer (nullable = true)
 |-- Wilderness_Area: integer (nullable = true)
 |-- Hillshade: string (nullable = true)



In [0]:
num_rows = treeCoverDF.count()
num_cols = len(treeCoverDF.columns)

print("Number of rows: {}".format(num_rows))
print("Number of columns: {}".format(num_cols))


Number of rows: 15120
Number of columns: 12


In [0]:
from pyspark.sql.functions import col

elevation_stats = treeCoverDF.select(col("elevation")).describe().toPandas()
elevation_range = (elevation_stats.loc[3, "elevation"], elevation_stats.loc[4, "elevation"])
elevation_mean = float(elevation_stats.loc[1, "elevation"])
elevation_stddev = float(elevation_stats.loc[2, "elevation"])

print("Range of elevation: {}".format(elevation_range))
print("Mean elevation: {}".format(elevation_mean))
print("Standard deviation of elevation: {}".format(elevation_stddev))


Range of elevation: ('1863', '3849')
Mean elevation: 2749.3225529100528
Standard deviation of elevation: 417.67818734804985


In [0]:
num_entries = treeCoverDF.filter((col("elevation") >= 2749.32) & ((col("Cover_Type") == 1) | (col("Cover_Type") == 2))).count()

print("Number of entries with elevation >= 2749.32 meters and Cover_Type 1 or 2: {}".format(num_entries))


Number of entries with elevation >= 2749.32 meters and Cover_Type 1 or 2: 3883


In [0]:
num_matching_entries = treeCoverDF.filter((col("elevation") >= 2749.32) & ((col("Cover_Type") == 1) | (col("Cover_Type") == 2))).count()
num_cover_type_entries = treeCoverDF.filter((col("Cover_Type") == 1) | (col("Cover_Type") == 2)).count()

percent_matching_entries = 100.0 * num_matching_entries / num_cover_type_entries

print("Percentage of entries with Cover_Type 1 or 2 and elevation at or above 2749.32 meters: {:.2f}%".format(percent_matching_entries))


Percentage of entries with Cover_Type 1 or 2 and elevation at or above 2749.32 meters: 89.88%


In [0]:
display(treeCoverDF.groupBy("Wilderness_Area").count())


Wilderness_Area,count
1,3597
3,6349
4,4675
2,499


In [0]:
# Read in the covertype.csv file
cover_type_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tmp/covertype.csv")

# Join the two DataFrames based on the Cover_Type column
joined_df = treeCoverDF.join(cover_type_df, on="outer")

# Group the DataFrame by the Cover Type Labels and calculate the average elevation
avg_elevation_df = joined_df.groupBy("Cover_Type_Label").avg("elevation").orderBy("Cover_Type_Label")

# Pass the resulting DataFrame to the display function to generate a bar chart visualization
display(avg_elevation_df)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-295471767776684> in <cell line: 5>()
      3 
      4 # Join the two DataFrames based on the Cover_Type column
----> 5 joined_df = treeCoverDF.join(cover_type_df, on="outer")
      6 
      7 # Group the DataFrame by the Cover Type Labels and calculate the average elevation

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in join(self, other, on, how)
   1623                 on = self._jseq([])
   1624             assert isinstance(how, str), "how should be a string"
->

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

# Define the schema for the u.data.csv file
u_data_schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("timestamp", IntegerType(), True)
])

# Define the schema for the u.item.csv file
u_item_schema = StructType([
    StructField("item_id", IntegerType(), True),
    StructField("movie_title", StringType(), True),
    StructField("release_date", StringType(), True),
    StructField("video_release_date", StringType(), True),
    StructField("IMDb_URL", StringType(), True),
    StructField("unknown", IntegerType(), True),
    StructField("Action", IntegerType(), True),
    StructField("Adventure", IntegerType(), True),
    StructField("Animation", IntegerType(), True),
    StructField("Childrens", IntegerType(), True),
    StructField("Comedy", IntegerType(), True),
    StructField("Crime", IntegerType(), True),
    StructField("Documentary", IntegerType(), True),
    StructField("Drama", IntegerType(), True),
    StructField("Fantasy", IntegerType(), True),
    StructField("Film_Noir", IntegerType(), True),
    StructField("Horror", IntegerType(), True),
    StructField("Musical", IntegerType(), True),
    StructField("Mystery", IntegerType(), True),
    StructField("Romance", IntegerType(), True),
    StructField("Sci_Fi", IntegerType(), True),
    StructField("Thriller", IntegerType(), True),
    StructField("War", IntegerType(), True),
    StructField("Western", IntegerType(), True)
])

# Read in the u.data.csv file using the specified schema and load it into a DataFrame
uDataDF = spark.read.format("csv").schema(u_data_schema).load("dbfs:/FileStore/tmp/u.data.csv")

# Read in the u.item.csv file using the specified schema and load it into a DataFrame, dropping unnecessary columns
uItemDF = spark.read.format("csv").schema(u_item_schema).load("dbfs:/FileStore/tmp/u.item.csv").drop("movie_title", "video_release_date", "IMDb_URL")

# Convert the release_date column to a DateType with the format yyyy-MM-dd
uItemDF = uItemDF.withColumn("release_date", uItemDF["release_date"].cast(StringType()))
uItemDF = uItemDF.withColumn("release_date", uItemDF["release_date"].cast(DateType()))

# Order the uDataDF DataFrame by the date column
uDataDF = uDataDF.orderBy("timestamp")


In [0]:
#recommendations can be made:

#Use IAM roles and policies to manage access to Databricks resources.
#Encrypt data at rest using AWS Key Management Service (KMS) and enable encryption in transit using SSL/TLS.
#Use network security best practices such as Virtual Private Cloud (VPC) and Security Groups to restrict access to Databricks clusters and workspaces.#

In [0]:
#  In 4-5 sentences, please explain what the Databricks API is used for at a high-level.


 # The Databricks API is a set of RESTful APIs that allows developers and data scientists to programmatically manage and interact with Databricks workspaces, clusters, jobs, and data. It can be used to automate common tasks such as creating and deleting clusters, running and monitoring jobs, uploading and downloading data, and managing user access and permissions. The API provides a convenient way to integrate Databricks with other tools and services and enables users to build custom applications and workflows that leverage the power of Databricks.

In [0]:
#Question 14: Conceptual Question - Explain an API Call
#In 4-5 sentences, please explain what this API call. Be sure to discuss some key attributes about the cluster

#The API call typically includes a set of parameters and an HTTP request method such as GET, POST, PUT, or DELETE. In the context of Databricks, an API call might be used to create, delete, or modify a cluster, submit a job, or retrieve data from a Databricks workspace. Some key attributes of a Databricks cluster that might be included in an API call include the number and type of nodes, the version of Spark being used, the cluster name, and any additional configuration settings such as auto-scaling and auto-termination policies.

In [0]:
from pyspark.sql.functions import count, col
from pyspark.sql.types import StructType, StructField, StringType

# Define the schema for the JSON data
json_schema = StructType([
  StructField("timestamp", StringType(), True),
  StructField("ip_address", StringType(), True),
  StructField("user_agent", StringType(), True)
])

# Read the JSON data into a DataFrame
json_data = spark.read.schema(json_schema).json("dbfs:/FileStore/tmp/u.item.csv")

# Group the data by IP address and count the occurrences
ip_counts = json_data.groupBy("ip_address").agg(count("*").alias("count")).orderBy(col("count").desc())

# Show the top 10 IP addresses by count
ip_counts.show(10)


+----------+-----+
|ip_address|count|
+----------+-----+
|      null| 1682|
+----------+-----+

